In [1]:
import cv2
import os
import numpy as np
from pathlib import Path

In [ ]:
# Paths
input_dir = Path("c:/Users/chskc/Desktop/Produtz/Task 2 - Change Detection Algorithm-20250630T074312Z-1-001/Task 2 - Change Detection Algorithm/input-images/")
output_dir = Path("task_2_output")
output_dir.mkdir(parents=True, exist_ok=True)

# Helper: Get all 'X.jpg' files
before_images = sorted([f for f in input_dir.glob("*.jpg") if "~2" not in f.name and "~3" not in f.name])

for before_path in before_images:
    base_name = before_path.stem
    after_path = input_dir / f"{base_name}~2.jpg"

    if not after_path.exists():
        print(f"Skipping {base_name}: after image not found.")
        continue

    # Read images
    before = cv2.imread(str(before_path))
    after = cv2.imread(str(after_path))

    if before is None or after is None:
        print(f"Error reading images for {base_name}.")
        continue

    # Convert to grayscale and blur
    gray_before = cv2.GaussianBlur(cv2.cvtColor(before, cv2.COLOR_BGR2GRAY), (5, 5), 0)
    gray_after = cv2.GaussianBlur(cv2.cvtColor(after, cv2.COLOR_BGR2GRAY), (5, 5), 0)

    # Absolute difference and threshold
    diff = cv2.absdiff(gray_before, gray_after)
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

    # Morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw bounding boxes on a copy of the after image
    annotated = after.copy()
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 500:  # Skip small noise
            continue
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(annotated, (x, y), (x+w, y+h), (0, 0, 255), 2)

    # Save result
    output_path = output_dir / f"{base_name}~3.jpg"
    cv2.imwrite(str(output_path), annotated)
    print(f"Processed and saved: {output_path.name}")


Processed and saved: 1~3.jpg
Processed and saved: 10~3.jpg
Processed and saved: 11~3.jpg
Processed and saved: 12~3.jpg
Processed and saved: 13~3.jpg
Processed and saved: 15~3.jpg
Processed and saved: 16~3.jpg
Processed and saved: 17~3.jpg
Processed and saved: 18~3.jpg
Processed and saved: 2~3.jpg
Processed and saved: 3~3.jpg
Processed and saved: 5~3.jpg
Processed and saved: 6~3.jpg
Processed and saved: 7~3.jpg
Processed and saved: 9~3.jpg
